[View in Colaboratory](https://colab.research.google.com/github/huyhoang17/Colab_Temporary/blob/master/[Test]RS_Implicit_ALS_lastfm.ipynb)

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe

In [2]:
!ls drive/My\ Drive/workspace/Recommender_System/lastfm

dataset  lastfm-dataset-360K.tar.gz  notebooks


In [0]:
# os.mkdir('drive/My Drive/workspace/Recommender_System/lastfm/dataset')

In [0]:
# import tarfile
# fname = 'drive/My Drive/workspace/Recommender_System/lastfm/lastfm-dataset-360K.tar.gz'
# if fname.endswith("tar.gz"):
#     tar = tarfile.open(fname, "r:gz")
#     tar.extractall('drive/My Drive/workspace/Recommender_System/lastfm/dataset')
#     tar.close()

In [3]:
!ls drive/My\ Drive/workspace/Recommender_System/lastfm/dataset/lastfm-dataset-360K

mbox_sha1sum.py  usersha1-artmbid-artname-plays.tsv
README.txt	 usersha1-profile.tsv


In [0]:
import os
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

In [5]:
ds_path = 'drive/My Drive/workspace/Recommender_System/lastfm/dataset'
lastfm_path = os.path.join(ds_path, 'lastfm-dataset-360K')
lastfm_path

'drive/My Drive/workspace/Recommender_System/lastfm/dataset/lastfm-dataset-360K'

In [6]:
!ls drive/My\ Drive/workspace/Recommender_System/lastfm/dataset/lastfm-dataset-360K

mbox_sha1sum.py  usersha1-artmbid-artname-plays.tsv
README.txt	 usersha1-profile.tsv


In [5]:
os.path.exists(lastfm_path)

True

In [0]:
raw_data = pd.read_table(os.path.join(lastfm_path, 'usersha1-artmbid-artname-plays.tsv'))

In [8]:
raw_data.head()

,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691


In [9]:
raw_data.shape

(17535654, 4)

In [10]:
raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'artist', 'plays']

# Drop rows with missing values
data = raw_data.dropna()

# Convert artists names into numerical IDs
data['user_id'] = data['user'].astype("category").cat.codes
data['artist_id'] = data['artist'].astype("category").cat.codes

# Create a lookup frame so we can get the artist names back in 
# readable form later.
item_lookup = data[['artist_id', 'artist']].drop_duplicates()
item_lookup['artist_id'] = item_lookup.artist_id.astype(str)

data = data.drop(['user', 'artist'], axis=1)

# Drop any rows that have 0 plays
data = data.loc[data.plays != 0]

# Create lists of all users, artists and plays
users = list(np.sort(data.user_id.unique()))
artists = list(np.sort(data.artist_id.unique()))
plays = list(data.plays)

# Get the rows and columns for our new matrix
rows = data.user_id.astype(int)
cols = data.artist_id.astype(int)

# Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
data_sparse.shape

(358868, 292363)

In [0]:
from tqdm import tqdm

In [0]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
    """ Implementation of Alternating Least Squares with implicit data. We iteratively
    compute the user (x_u) and item (y_i) vectors using the following formulas:

    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))

    Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix

        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.

        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors

        lambda_val (float): Regularization value

        features (int): How many latent features we want to compute.

    Returns:     
        X (csr_matrix): user vectors of size users-by-features

        Y (csr_matrix): item vectors of size items-by-features
     """

    # Calculate the foncidence for each value in our data
    confidence = sparse_data * alpha_val

    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape

    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)

    I = sparse.eye(features)
    lI = lambda_val * I
    
    # Start main loop. For each iteration we first compute X and then Y
    for i in range(iterations):
        print('iteration %d of %d' % (i + 1, iterations))
        
        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Loop through all users
        for u in tqdm(range(user_size)):

            # Get the user row.
            u_row = confidence[u,:].toarray() 

            # Calculate the binary preference p(u)
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

    
        for i in tqdm(range(item_size)):

            # Get the item column and transpose it.
            i_row = confidence[:,i].T.toarray()

            # Calculate the binary preference p(i)
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

In [18]:
user_vecs, item_vecs = implicit_als(data_sparse, iterations=1, features=20, alpha_val=40)

iteration 1 of 1


  0%|          | 310/358868 [01:14<23:50:21,  4.18it/s]

KeyboardInterrupt: ignored

In [28]:
!apt install cuda
!pip install implicit==0.3.8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  acpid adwaita-icon-theme at-spi2-core ca-certificates-java cuda-9-2
  cuda-compiler-9-2 cuda-cublas-dev-9-2 cuda-cufft-dev-9-2 cuda-curand-dev-9-2
  cuda-cusolver-dev-9-2 cuda-cusparse-dev-9-2 cuda-demo-suite-9-2
  cuda-documentation-9-2 cuda-drivers cuda-libraries-dev-9-2 cuda-npp-dev-9-2
  cuda-nsight-9-2 cuda-nvgraph-dev-9-2 cuda-nvml-dev-9-2 cuda-nvprune-9-2
  cuda-nvrtc-dev-9-2 cuda-nvvp-9-2 cuda-runtime-9-2 cuda-samples-9-2
  cuda-toolkit-9-2 cuda-tools-9-2 cuda-visual-tools-9-2 dbus
  dconf-gsettings-backend dconf-service default-jre default-jre-headless dkms
  dmsetup fakeroot fontconfig fonts-dejavu-core fonts-dejavu-extra freeglut3
  freeglut3-dev glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas gtk-update-icon-cache
  hicolor-icon-theme humanity-icon-theme java-common keyboard-configu

In [0]:
import implicit

In [30]:
data.head()

,plays,user_id,artist_id
0,1099,0,90933
1,897,0,185367
2,717,0,106704
3,706,0,155241
4,691,0,220128


In [0]:
raw_data = pd.read_table(os.path.join(lastfm_path, 'usersha1-artmbid-artname-plays.tsv'))
raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'artist', 'plays']

# Drop NaN columns
data = raw_data.dropna()
data = data.copy()

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

In [32]:
data.head()

,user,artist,plays,user_id,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,90933
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,0,185367
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,0,106704
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,0,155241
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,0,220128


In [0]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

In [34]:
sparse_item_user

<292363x358868 sparse matrix of type '<class 'numpy.float64'>'
	with 17535375 stored elements in Compressed Sparse Row format>

In [35]:
sparse_user_item

<358868x292363 sparse matrix of type '<class 'numpy.float64'>'
	with 17535375 stored elements in Compressed Sparse Row format>

In [36]:
data['user_id'].shape, data['artist_id'].shape

((17535450,), (17535450,))

In [37]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


In [25]:
model

In [0]:
# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [41]:
type(data_conf)

scipy.sparse.csr.csr_matrix

In [0]:
data_conf

In [39]:
model.fit(data_conf)


  0%|          | 0/20 [00:00<?, ?it/s]
  2%|▎         | 0.5/20 [00:00<00:19,  1.01it/s]
  5%|▌         | 1.0/20 [00:01<00:24,  1.31s/it]
  8%|▊         | 1.5/20 [00:02<00:22,  1.21s/it]
 10%|█         | 2.0/20 [00:03<00:26,  1.45s/it]
 12%|█▎        | 2.5/20 [00:03<00:23,  1.31s/it]
 15%|█▌        | 3.0/20 [00:06<00:45,  2.66s/it]
 18%|█▊        | 3.5/20 [00:06<00:35,  2.17s/it]
 20%|██        | 4.0/20 [00:09<00:52,  3.25s/it]
 22%|██▎       | 4.5/20 [00:10<00:39,  2.58s/it]
 25%|██▌       | 5.0/20 [00:13<00:53,  3.54s/it]
 28%|██▊       | 5.5/20 [00:13<00:40,  2.77s/it]
 30%|███       | 6.0/20 [00:16<00:51,  3.67s/it]
 32%|███▎      | 6.5/20 [00:17<00:38,  2.88s/it]
 35%|███▌      | 7.0/20 [00:19<00:48,  3.75s/it]
 38%|███▊      | 7.5/20 [00:20<00:36,  2.92s/it]
 40%|████      | 8.0/20 [00:23<00:45,  3.78s/it]
 42%|████▎     | 8.5/20 [00:23<00:33,  2.95s/it]
 45%|████▌     | 9.0/20 [00:26<00:41,  3.79s/it]
 48%|████▊     | 9.5/20 [00:27<00:31,  2.95s/it]
 50%|█████     | 10.0/20 [00:

In [0]:
# Find the 10 most similar to Jay-Z
item_id = 147068 #Jay-Z
n_similar = 10

In [43]:
model.use_gpu

True

In [0]:
# Get the user and item vectors from our trained model
user_vecs = model.user_factors
item_vecs = model.item_factors

In [45]:
user_vecs.shape

(358868, 32)

In [46]:
item_vecs.shape

(292363, 32)

In [47]:
item_vecs

array([[-5.6301883e-06, -1.2624863e-05,  1.2741190e-05, ...,
        -1.3160368e-05, -1.6411353e-05,  3.6287861e-06],
       [ 2.5084019e-03, -6.4977095e-03,  5.4144632e-04, ...,
        -4.3445062e-03, -7.8636892e-03, -3.9436668e-04],
       [-2.1971375e-04,  9.6211843e-05,  1.1375421e-04, ...,
         9.6576601e-05, -2.1186795e-04, -8.5151136e-05],
       ...,
       [-1.6205923e-03,  3.3260535e-03,  2.2060887e-03, ...,
        -1.4680428e-04,  6.0867544e-05, -5.7063857e-04],
       [ 7.6797773e-04,  1.5353750e-03,  1.6126351e-03, ...,
         3.8693968e-04,  2.5001117e-03,  1.0965897e-03],
       [-1.9997684e-03, -2.2198183e-03, -9.8850264e-04, ...,
        -7.5788812e-06, -1.7106013e-03, -1.2740255e-03]], dtype=float32)

In [0]:
# Calculate the vector norms
item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))

In [49]:
item_norms.shape

(292363,)

In [50]:
item_norms

array([8.2388506e-05, 2.0304542e-02, 6.6063774e-04, ..., 1.0225646e-02,
       6.5628961e-03, 7.1776020e-03], dtype=float32)

In [0]:
scores = item_vecs.dot(item_vecs[item_id]) / item_norms

In [52]:
scores

array([ 0.00999367, -0.01358925, -0.00674658, ...,  0.01309817,
        0.02638995, -0.02531872], dtype=float32)

In [53]:
max(scores)

0.04776986

In [54]:
min(scores)

-0.03077719

In [0]:
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]

In [56]:
top_idx

array([195384, 205889,   3845, 263308, 223171, 261970, 147068,   2721,
       175905, 238216])

In [0]:
similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])

In [58]:
similar

[(147068, 0.99999994),
 (238216, 0.9970172),
 (2721, 0.99214715),
 (205889, 0.9921287),
 (175905, 0.9916901),
 (261970, 0.99064505),
 (3845, 0.98762625),
 (223171, 0.9828211),
 (263308, 0.9827492),
 (195384, 0.9824248)]

In [59]:
# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(data.artist.loc[data.artist_id == idx].iloc[0])

jay-z
snoop dogg
2pac
outkast
lupe fiasco
the roots
50 cent
rjd2
the streets
n*e*r*d


In [60]:
data.head()

,user,artist,plays,user_id,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,90933
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897,0,185367
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717,0,106704
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706,0,155241
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,0,220128


In [0]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------
def recommend(user_id, sparse_user_item, user_vecs, item_vecs, num_items=10):
    """The same recommendation function we used before"""

    user_interactions = sparse_user_item[user_id,:].toarray()

    user_interactions = user_interactions.reshape(-1) + 1
    user_interactions[user_interactions > 1] = 0

    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled

    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    artists = []
    scores = []

    for idx in item_idx:
        artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'artist': artists, 'score': scores})

    return recommendations

In [0]:
# Get the trained user and item vectors. We convert them to 
# csr matrices to work with our previous recommend function.
user_vecs = sparse.csr_matrix(model.user_factors)
item_vecs = sparse.csr_matrix(model.item_factors)

In [0]:
# Create recommendations for user with id 2025
user_id = 2025

In [0]:
recommendations = recommend(user_id, sparse_user_item, user_vecs, item_vecs)

In [65]:
print(recommendations)

                   artist     score
0               sam cooke  1.000000
1                the band  0.995662
2                    ween  0.995482
3  sly & the family stone  0.981407
4            brian wilson  0.975953
5            otis redding  0.973579
6         thelonious monk  0.972716
7         curtis mayfield  0.972609
8          duke ellington  0.971102
9                al green  0.970876


In [0]:
# Using recommend method
# Create recommendations for user with id 2025
user_id = 2025

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

In [67]:
recommended

[(228211, 1.2123001),
 (252327, 1.2042192),
 (279110, 1.2038832),
 (237704, 1.1776652),
 (56148, 1.1675047),
 (205656, 1.1630838),
 (265267, 1.1614755),
 (78883, 1.1612756),
 (101331, 1.1584694),
 (15100, 1.1580477)]

In [0]:
artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

In [69]:
# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})
print(recommendations)

                   artist     score
0               sam cooke  1.212300
1                the band  1.204219
2                    ween  1.203883
3  sly & the family stone  1.177665
4            brian wilson  1.167505
5            otis redding  1.163084
6         thelonious monk  1.161476
7         curtis mayfield  1.161276
8          duke ellington  1.158469
9                al green  1.158048


In [0]:
add = recommendations['score'][0] - 1

In [72]:
print(recommendations['score'] - add)

0    1.000000
1    0.978973
2    0.973336
3    0.964416
4    0.964408
5    0.962556
6    0.958192
7    0.951517
8    0.950474
9    0.947800
Name: score, dtype: float64


In [0]:
# https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe